In [ ]:
import os
import time
import logging
import fiona
import rasterio
import numpy as np
import pandas as pd
import xarray as xr
import rioxarray as rxr
import geopandas as gpd
import shapely.speedups
import matplotlib.pyplot as plt
from glob import glob
from rasterio.plot import show
from rasterio.plot import show_hist
from omegaconf import OmegaConf
from omegaconf.listconfig import ListConfig
from multiprocessing import Pool, cpu_count

# import cuspatial
import tensorflow as tf
from pathlib import Path
from shapely.geometry import box
from skimage.transform import resize
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import mean_squared_error

shapely.speedups.enable()

In [ ]:
data_regex = '/adapt/nobackup/people/mcarrol2/LCLUC_Senegal/ForKonrad/Tappan01_WV02_20110430_M1BS_103001000A27E100_data.tif'
chm_regex = '/adapt/nobackup/people/jacaraba/development/vhr-cnn-chm/data/cnn_chm_v2/Tappan01_WV02_20110430_M1BS_103001000A27E100_data.2m_chm.tif'

In [ ]:
data_filenames = glob(data_regex)
chm_filenames = glob(chm_regex)

In [ ]:
for dataf, chmf in zip(data_filenames, chm_filenames):
    
    data_src = rasterio.open(dataf)
    chm_src = rasterio.open(chmf)
    
    fig, (axrgb, axhist) = plt.subplots(1, 2, figsize=(14,7))
    # show(data_src, ax=axrgb)
    show((data_src, 7), ax=axrgb, cmap='Greens', title='nir channel')
    show_hist(
        data_src, bins=50, histtype='stepfilled', lw=0.0,
        stacked=False, alpha=0.3, ax=axhist
    )
    plt.show()
    
    fig, (axrgb, axhist) = plt.subplots(1, 2, figsize=(14,7))
    # show(data_src, ax=axrgb)
    show(chm_src, ax=axrgb, cmap='Greens', title='nir channel')
    show_hist(
        chm_src, bins=50, histtype='stepfilled', lw=0.0,
        stacked=False, alpha=0.3, ax=axhist
    )
    plt.show()

In [ ]:
# 1. h_can values of the training - 
# 

In [ ]:
import folium
from folium import Map, TileLayer, GeoJson, LayerControl, Icon, \
    Marker, features, Figure, CircleMarker, plugins

In [ ]:
basemaps = {
       'Google Terrain' : TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=p&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Terrain',
        overlay = False,
        control = True
       ),
        'basemap_gray' : TileLayer(
            tiles="http://services.arcgisonline.com/ArcGIS/rest/services/Canvas/World_Light_Gray_Base/MapServer/tile/{z}/{y}/{x}",
            opacity=1,
            name="World gray basemap",
            attr="ESRI",
            overlay=False
        ),
        'Imagery' : TileLayer(
            tiles='https://services.arcgisonline.com/arcgis/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
            opacity=1,
            name="World Imagery",
            attr="ESRI",
            overlay=False
        ),
        'ESRINatGeo' : TileLayer(
            tiles='https://server.arcgisonline.com/ArcGIS/rest/services/NatGeo_World_Map/MapServer/tile/{z}/{y}/{x}',
            opacity=1,
            name='ESRI NatGeo',
            attr='ESRI',
            overlay=False
        )
}

In [ ]:
# Map the Layers
Map_Figure = Figure(width=1000, height=400)

foliumMap = Map(
    #location=(atl08_gdf.lat.mean(), atl08_gdf.lon.mean()),
    zoom_start=8,
    control_scale=True,
    tiles=None
)
Map_Figure.add_child(foliumMap)

basemaps['Imagery'].add_to(foliumMap)
LayerControl().add_to(foliumMap)
plugins.Fullscreen().add_to(foliumMap)
plugins.Geocoder().add_to(foliumMap)
plugins.MousePosition().add_to(foliumMap)

# Add marker of imagery based on year options
# Add marker of imagery and zoom into overlay raster
# Add marker of imagery and push on and off landcover layers


foliumMap

In [ ]:
from glob import glob
import rasterio as rio
from pyproj import Transformer

In [ ]:
#data_regex = [
#    '/adapt/nobackup/people/mwooten3/Senegal_LCLUC/VHR/CAS/M1BS/*.tif',
#    '/adapt/nobackup/people/mwooten3/Senegal_LCLUC/VHR/ETZ/M1BS/*.tif',
#    '/adapt/nobackup/people/mwooten3/Senegal_LCLUC/VHR/SRV/M1BS/*.tif'
#]
#data_regex = ['/gpfsm/ccds01/nobackup/projects/3sl/data/Tappan/*_data.tif']
data_regex = ['/adapt/nobackup/projects/ilab/projects/Senegal/CNN_CHM/v4/output/*_data*.tif']

In [ ]:
filenames = []
if isinstance(data_regex, list) or isinstance(data_regex, ListConfig):
    for regex in data_regex:
        filenames.extend(glob(regex))
else:
    filenames = glob(data_regex)
print(len(filenames))

#filenames = ['/gpfsm/ccds01/nobackup/projects/3sl/data/VHR/CAS/M1BS/WV02_20210209_M1BS_10300100B3074C00-toa-cog.tif']
#filenames = ['/gpfsm/ccds01/nobackup/projects/3sl/data/VHR/CAS/M1BS/WV02_20210209_M1BS_10300100B3074C00-toa.tif',
#             '/gpfsm/ccds01/nobackup/projects/3sl/data/VHR/CAS/M1BS/WV02_20210209_M1BS_10300100B3074C00-toa-cog.tif',
#             '/gpfsm/ccds01/nobackup/projects/3sl/data/Tappan/Tappan09_WV02_20191127_M1BS_103001009D5BF100_data-cog.tif',
#            '/gpfsm/ccds01/nobackup/projects/3sl/data/Tappan/Tappan09_WV02_20191127_M1BS_103001009D5BF100_data.tif']

In [ ]:
"""
dst_crs = 'EPSG:4326'

for in_path in filenames:
    
    with rio.open(in_path) as src:

        img = src.read([5,2,7])
        print(src.crs)


        src_crs = src.crs['init'].upper()
        min_lon, min_lat, max_lon, max_lat = src.bounds
        print(min_lon, min_lat, max_lon, max_lat)

    ## Conversion from UTM to WGS84 CRS
    bounds_orig = [[min_lat, min_lon], [max_lat, max_lon]]

    bounds_fin = []

    for item in bounds_orig:   
        #converting to lat/lon
        lat = item[0]
        lon = item[1]

        proj = Transformer.from_crs(int(src_crs.split(":")[1]), int(dst_crs.split(":")[1]), always_xy=True)

        lon_n, lat_n = proj.transform(lon, lat)

        bounds_fin.append([lat_n, lon_n])

    # Finding the centre latitude & longitude    
    centre_lon = bounds_fin[0][1] + (bounds_fin[1][1] - bounds_fin[0][1])/2
    centre_lat = bounds_fin[0][0] + (bounds_fin[1][0] - bounds_fin[0][0])/2
    print(centre_lon, centre_lat)
    
    #folium.Marker([centre_lat, centre_lon], popup="Camp Muir").add_to(foliumMap)
    # Overlay raster (RGB) called img using add_child() function (opacity and bounding box set)
    foliumMap.add_child(folium.raster_layers.ImageOverlay(img.transpose(1, 2, 0), opacity=.7, 
                                 bounds = bounds_fin))
"""